# Window Functions
Window functions perform calculations across a set of table rows that are somehow related to the current row. Unlike aggregate functions, window functions do not cause rows to become grouped into a single output row. This means you can retain the individual rows while performing an aggregation across a set of them.  

![alt text](Agg_Window_Funct.JPG)


### Syntax
Window functions can be defined in the SELECT clause of a query.  
```sql
SELECT
    column1,
    column2,
    window_function(column3) OVER (
        PARTITION BY column1
        ORDER BY column2
        RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS column3
FROM
    table_name;
```

## Difference between GROUP BY Aggregation and Window Functions
GROUP BY Aggregations return a single value for each group of rows. Window functions return a single value for each row.  
Find out the average salary for male and female employees.    


Lets see the result of following queries. (USE test03 database)  
```sql
SELECT gender, AVG(sal.salary) AS avg_salary
FROM employee_demographics dem
JOIN employee_salary sal
ON dem.employee_id = sal.employee_id
GROUP BY gender;
```
<hr>

```sql
SELECT dem.first_name, gender, AVG(sal.salary) OVER(PARTITION BY gender)
FROM employee_demographics dem
JOIN employee_salary sal
ON dem.employee_id = sal.employee_id;
```

## List of Window Functions
1. [ROW_NUMBER()](#ROW_NUMBER())
2. [RANK()](#RANK())
3. [DENSE_RANK()](#DENSE_RANK())
4. [PERCENT_RANK()](#PERCENT_RANK())
5. [CUME_DIST()](#CUME_DIST())
6. [NTILE()](#NTILE())
7. [LAG()](#LAG())
8. [LEAD()](#LEAD())
9. [FIRST_VALUE()](#FIRST_VALUE())
10. [LAST_VALUE()](#LAST_VALUE())
11. [NTH_VALUE()](#NTH_VALUE())


### ROW_NUMBER()
The ROW_NUMBER() function assigns a unique integer value to each row within the partition of a result set. The row number starts with 1 for the first row in each partition.  

You can partition by gender or dept_id and order by salary.  

For example:
```sql
SELECT dem.employee_id, dem.first_name, dem.gender, sal.salary, 
ROW_NUMBER() OVER()     AS row_num
FROM employee_demographics dem
JOIN employee_salary sal
ON sal.employee_id = dem.employee_id;
```


## RANK()
The difference between RANK() and ROW_NUMBER() is that RANK() will assign the same value to the rows that have the same value in the ordered column. The next row will have a rank value that is incremented by the number of rows with the same value.

For example:
```sql
SELECT dem.employee_id, dem.first_name, dem.gender, sal.salary, 
ROW_NUMBER() OVER(PARTITION BY gender ORDER BY sal.salary)  AS row_num,
RANK() OVER(PARTITION BY gender ORDER BY sal.salary)  AS rank_num
FROM employee_demographics dem
JOIN employee_salary sal
ON sal.employee_id = dem.employee_id;
```

## DENSE_RANK()
The DENSE_RANK() function is similar to the RANK() function. The difference is that the DENSE_RANK() function does not leave gaps in the ranking sequence when there are ties.

For example:
```sql
SELECT dem.employee_id, dem.first_name, dem.gender, sal.salary, 
ROW_NUMBER() OVER(PARTITION BY gender ORDER BY sal.salary)  AS row_num,
RANK() OVER(PARTITION BY gender ORDER BY sal.salary)  AS rank_num,
DENSE_RANK() OVER(PARTITION BY gender ORDER BY sal.salary)  AS dense_rank_num
FROM employee_demographics dem
JOIN employee_salary sal
ON sal.employee_id = dem.employee_id;
```


## NTILE()
The NTILE() function divides the result set into a specified number of approximately equal groups. The groups are numbered, starting at one.

For example:
```sql
SELECT dem.employee_id, dem.first_name, dem.gender, sal.salary, 
NTILE(4) OVER(PARTITION BY gender ORDER BY sal.salary)  AS quartile
FROM employee_demographics dem
JOIN employee_salary sal
ON sal.employee_id = dem.employee_id;
```

## LAG(column, offset, default_value)
The LAG() function is used to access data from a previous row in the same result set without the use of a self-join. The LAG() function returns the value of column from the previous row that is offset rows before the current row. If there is no such row, the default_value is returned.

For example:
```sql
SELECT dem.employee_id, dem.first_name, dem.gender, sal.salary, 
LAG(sal.salary,1,0) OVER(PARTITION BY gender ORDER BY sal.salary)  AS prev_val
FROM employee_demographics dem
JOIN employee_salary sal
ON sal.employee_id = dem.employee_id;
```

## LEAD(column, offset, default_value)
The LEAD() function is used to access data from a subsequent row in the same result set without the use of a self-join. The LEAD() function returns the value of column from the next row that is offset rows after the current row. If there is no such row, the default_value is returned.

For example:
```sql
SELECT dem.employee_id, dem.first_name, dem.gender, sal.salary, 
LEAD(sal.salary,1,0) OVER(PARTITION BY gender ORDER BY sal.salary)  AS next_val
FROM employee_demographics dem
JOIN employee_salary sal
ON sal.employee_id = dem.employee_id;
```


<hr>

# Frame Specifying Clause
The frame_spec clause defines the window frame over which the window function operates. 

**Syntax**  
```sql
[ ROWS | RANGE ]
BETWEEN frame_start AND frame_end
```

**Default Frame is:**
```sql
RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
```  

**Frame Boundaries**  
 - UNBOUNDED PRECEDING: The frame starts at the first row of the partition.
 - N PRECEDING: The frame starts N rows before the current row.
 - CURRENT ROW: The frame starts or ends at the current row.
 - N FOLLOWING: The frame starts N rows after the current row.
 - UNBOUNDED FOLLOWING: The frame ends at the last row of the partition.

Lets use it inside the window functions LAST_VALUE() and FIRST_VALUE().

<hr>

## FIRST_VALUE()
The FIRST_VALUE() function returns the first value in an ordered set of values. The frame_spec clause is used to define the window frame over which the window function operates.

For example:
```sql
SELECT dem.employee_id, dem.first_name, dem.gender, sal.salary, 
RANK() OVER(PARTITION BY gender ORDER BY sal.salary)  AS rank_num,
FIRST_VALUE(sal.first_name) OVER(PARTITION BY gender ORDER BY sal.salary
	RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
)  AS first_val
FROM employee_demographics dem
JOIN employee_salary sal
ON sal.employee_id = dem.employee_id;
```

## LAST_VALUE()
The LAST_VALUE() function returns the last value in an ordered set of values. The frame_spec clause is used to define the window frame over which the window function operates.

For example:
```sql
SELECT dem.employee_id, dem.first_name, dem.gender, sal.salary, 
RANK() OVER(PARTITION BY gender ORDER BY sal.salary)  AS rank_num,
LAST_VALUE(sal.first_name) OVER(PARTITION BY gender ORDER BY sal.salary
	RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
)  AS last_val
FROM employee_demographics dem
JOIN employee_salary sal
ON sal.employee_id = dem.employee_id;
```
